In [1]:
import numpy as np
import pandas as pd
import pickle
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

import requests
from sentence_transformers import SentenceTransformer, util

with open('X_scaler_10p_v0.pkl', 'rb') as file:
    scaler = pickle.load(file)

with open('xgboost_embedding_10p_v0.pkl', 'rb') as file:
    best_model = pickle.load(file)

import torch.nn.functional as F
from sentence_transformers import SentenceTransformer

revision = None  # Replace with the specific revision to ensure reproducibility if the model is updated.
model = SentenceTransformer("avsolatorio/GIST-small-Embedding-v0", revision=revision)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Alan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Alan\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Alan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Alan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\Alan\Desktop\dva_project_test\tabpy_venv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
C:\Users

In [2]:
bdpt_dict={}
bdpt_dict[30]='SHOULDER'
bdpt_dict[31]='UPPERTRUNK'
bdpt_dict[32]='ELBOW'
bdpt_dict[33]='LOWERARM'
bdpt_dict[34]='WRIST'
bdpt_dict[35]='KNEE'
bdpt_dict[36]='LOWERLEG'
bdpt_dict[37]='ANKLE'
bdpt_dict[38]='PUBICREGION'
bdpt_dict[75]='HEAD'
bdpt_dict[76]='FACE'
bdpt_dict[77]='EYEBALL'
bdpt_dict[78]='UPPERTRUNK(OLD)'
bdpt_dict[79]='LOWERTRUNK'
bdpt_dict[80]='UPPERARM'
bdpt_dict[81]='UPPERLEG'
bdpt_dict[82]='HAND'
bdpt_dict[83]='FOOT'
bdpt_dict[88]='MOUTH'
bdpt_dict[89]='NECK'
bdpt_dict[90]='LOWERARM(OLD)'
bdpt_dict[91]='LOWERLEG(OLD)'
bdpt_dict[92]='FINGER'
bdpt_dict[93]='TOE'
bdpt_dict[94]='EAR'

In [3]:
def create_df(X):

    sample_df_single_record=pd.DataFrame(X,columns=['Age', 'Sex','Location','Product_1' ,'activity_at_injury', 'object_involved','injury_mechanism','More_details'])

    sample_df_single_record["Age"] = pd.to_numeric(sample_df_single_record["Age"])
    sample_df_single_record["Sex"] = pd.to_numeric(sample_df_single_record["Sex"])
    sample_df_single_record["Location"] = pd.to_numeric(sample_df_single_record["Location"])
    sample_df_single_record["Product_1"] = pd.to_numeric(sample_df_single_record["Product_1"])

    sample_df_single_record['Narrative']=sample_df_single_record["activity_at_injury"].astype(str) + ' '+sample_df_single_record["injury_mechanism"].astype(str)+ ' ' + sample_df_single_record["object_involved"].astype(str)+' '+sample_df_single_record["More_details"].astype(str)

    Embedding_df=pd.DataFrame(model.encode(sample_df_single_record['Narrative'], convert_to_tensor=True).numpy())
    Embedding_df.columns=Embedding_df.columns.astype(str)

    sample_df_single_record_2=sample_df_single_record[["Age","Sex","Location","Product_1"]]

    return Embedding_df,sample_df_single_record_2

In [4]:
def gen_prediction(bdpt_dict,data_ready,scaler,best_model): #,sematic_distance_bert


    prob_out={}

    k=0

    for i in bdpt_dict:
        data_ready.at[0,'Body_Part']=i

        X_test_scaled = scaler.transform(data_ready.select_dtypes(include=['number']))
        X_test_final=pd.DataFrame(X_test_scaled, columns=data_ready.select_dtypes(include=['number']).columns)
        y_prob = best_model.predict_proba(X_test_final)[:, 1][0]
        #print(y_prob)
        prob_out[str(i)]=y_prob
        k+=1

    return prob_out

In [5]:
def Predict_Body_parts_Prob(Age, Sex, Location, Product_1, activity_at_injury, object_involved, injury_mechanism, More_details):
 
    X = np.column_stack([Age, Sex, Location, Product_1, activity_at_injury, object_involved, injury_mechanism, More_details])
    embedding_df, sample_df_single_record_2 = create_df(X)
    data = {"Body_Part": [25]}
    df_body = pd.DataFrame(data)
    data_ready = pd.concat([sample_df_single_record_2[['Age', 'Sex']], df_body, sample_df_single_record_2[['Location', 'Product_1']], embedding_df], axis=1)
    output = gen_prediction(bdpt_dict, data_ready, scaler, best_model)

    probabilities = [float(value) for value in output.values()]
    
    return probabilities

In [10]:
Predict_Body_parts_Prob(25, 1,1,1202 ,"fall walk", "bike","","fall from bike when riding")

[0.19397300481796265,
 0.7552107572555542,
 0.8747296929359436,
 0.8099174499511719,
 0.6968040466308594,
 0.4638219177722931,
 0.6632651686668396,
 0.5643863677978516,
 0.7467895746231079,
 0.9490506052970886,
 0.7697350382804871,
 0.8380334973335266,
 0.8356714844703674,
 0.8356714844703674,
 0.7234531044960022,
 0.7650008201599121,
 0.7650008201599121,
 0.291553258895874,
 0.6762338280677795,
 0.3676290810108185,
 0.3676290810108185,
 0.3676290810108185,
 0.3676290810108185,
 0.19985206425189972,
 0.18288663029670715]

In [7]:
from tabpy.tabpy_tools.client import Client

deploy = Client('http://localhost:9004/')
deploy.set_credentials(username='dva023', password='YL8bar-_3.jXGFet')
deploy.deploy('tabpy_v5_2', Predict_Body_parts_Prob, 'tabpy v5.2 database test', override=True)